<a href="https://colab.research.google.com/github/rjshrd/myrepos/blob/master/Dyslexia_binary_NNClassifier_and_Self_Supervised_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# read the data sets - obtained from Kaggle
reference: "https://www.kaggle.com/datasets/luzrello/dyslexia/data"

In [1]:
import pandas as pd
desktopData= pd.read_csv("/content/Dyt-desktop.csv", sep = ';', encoding = 'utf-8')
tabletData = pd.read_csv("/content/Dyt-tablet.csv", sep = ';', encoding = 'utf-8')

In [2]:
# Both the data have same columns and attributes, therefore combininig it
ifNUll= tabletData.isnull().any()
i = 0
for state in ifNUll :
    if(state):
        tabletData[ifNUll.index[i]].fillna(round(tabletData[ifNUll.index[i]].mean() , 4), inplace=True)
    i = i + 1

dyslexia_df = pd.concat([desktopData, tabletData], axis=0).drop_duplicates()

**Clean and Encode the categorical values**

In [3]:
def cleanData(data) :
    for col in data.columns.values:
        data[col] = data[col].astype('string')
    #----------
    for col in data.columns.values:
        data[col] = data[col].astype('float',errors = 'ignore')
    #-----------
    data['Gender']=data.Gender.map({'Male': 1, 'Female': 2})
    data['Dyslexia']=data.Dyslexia.map({'No': 0, 'Yes': 1})
    data['Nativelang']=data.Nativelang.map({'No': 0, 'Yes': 1})
    data['Otherlang']=data.Otherlang.map({'No': 0, 'Yes': 1})

In [4]:
cleanData(dyslexia_df)

In [5]:
#dyslexia_df

print(dyslexia_df.groupby('Gender').size())
print(dyslexia_df.groupby('Dyslexia').size())
print(dyslexia_df.groupby('Nativelang').size())
print(dyslexia_df.groupby('Otherlang').size())

Gender
1    2546
2    2493
dtype: int64
Dyslexia
0    4499
1     540
dtype: int64
Nativelang
0    2725
1    2314
dtype: int64
Otherlang
0    1967
1    3072
dtype: int64


In [6]:
cols_with_missing = [col for col in dyslexia_df.columns if dyslexia_df[col].isnull().any()]

# Drop columns desktop data
dyslexia_df1 = dyslexia_df.drop(cols_with_missing, axis=1)
dyslexia_df1

,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,1,0,1,7.0,10.0,10.0,0.0,10.0,1.0,0.0,...,0.0,0.000000,0.00,17.0,2.0,0.0,2.0,0.117647,0.000000,0
1,2,1,1,13.0,12.0,12.0,0.0,12.0,1.0,0.0,...,4.0,0.114286,0.00,26.0,2.0,2.0,2.0,0.076923,0.076923,1
2,2,0,1,7.0,6.0,6.0,0.0,6.0,1.0,0.0,...,4.0,0.114286,0.00,26.0,1.0,3.0,1.0,0.038462,0.115385,0
3,2,0,1,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.00,1.0,0.0,0.0,0.0,0.000000,0.000000,0
4,2,0,1,8.0,4.0,4.0,0.0,4.0,1.0,0.0,...,1.0,25.000000,0.05,26.0,2.0,2.0,2.0,0.076923,0.076923,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,1,1,0,17.0,13.0,13.0,0.0,13.0,1.0,0.0,...,35.0,0.114286,0.00,26.0,4.0,0.0,26.0,0.153846,0.000000,0
1391,2,1,1,17.0,9.0,9.0,0.0,9.0,1.0,0.0,...,35.0,0.114286,0.00,26.0,4.0,0.0,26.0,0.153846,0.000000,0
1392,1,1,1,17.0,10.0,10.0,0.0,10.0,1.0,0.0,...,35.0,0.114286,0.00,27.0,3.0,2.0,27.0,0.111111,0.074074,0
1393,2,1,1,17.0,11.0,11.0,0.0,11.0,1.0,0.0,...,35.0,0.114286,0.00,26.0,4.0,0.0,26.0,0.153846,0.000000,0


In [7]:
# Common attributes for columns
commonalityColumns = ['Gender','Nativelang','Otherlang','Age','Dyslexia']
for i in  range(30):
    if((i>=0 and i<12) or (i>=13 and i<17) or i==21 or i==22 or i==29):
        commonalityColumns.append('Clicks'+str(i+1))
        commonalityColumns.append('Hits'+str(i+1))
        commonalityColumns.append('Misses'+str(i+1))
        commonalityColumns.append('Score'+str(i+1))
        commonalityColumns.append('Accuracy'+str(i+1))
        commonalityColumns.append('Missrate'+str(i+1))

dyslexia_df1=dyslexia_df1.loc[:,commonalityColumns]
dyslexia_df1

,Gender,Nativelang,Otherlang,Age,Dyslexia,Clicks1,Hits1,Misses1,Score1,Accuracy1,...,Misses23,Score23,Accuracy23,Missrate23,Clicks30,Hits30,Misses30,Score30,Accuracy30,Missrate30
0,1,0,1,7.0,0,10.0,10.0,0.0,10.0,1.0,...,1.0,3.0,0.750000,0.250000,14.0,1.0,2.0,1.0,0.071429,0.142857
1,2,1,1,13.0,1,12.0,12.0,0.0,12.0,1.0,...,1.0,5.0,0.833333,0.166667,17.0,2.0,2.0,2.0,0.117647,0.117647
2,2,0,1,7.0,0,6.0,6.0,0.0,6.0,1.0,...,2.0,3.0,0.600000,0.400000,17.0,1.0,3.0,1.0,0.058824,0.176471
3,2,0,1,7.0,0,0.0,0.0,0.0,0.0,0.0,...,1.0,3.0,0.750000,0.250000,8.0,0.0,1.0,0.0,0.000000,125.000000
4,2,0,1,8.0,0,4.0,4.0,0.0,4.0,1.0,...,3.0,2.0,0.400000,0.600000,17.0,0.0,4.0,0.0,0.000000,0.235294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,1,1,0,17.0,0,13.0,13.0,0.0,13.0,1.0,...,0.0,7.0,1.000000,0.000000,21.0,4.0,0.0,21.0,0.190476,0.000000
1391,2,1,1,17.0,0,9.0,9.0,0.0,9.0,1.0,...,1.0,6.0,0.833333,0.166667,21.0,3.0,1.0,21.0,0.142857,0.047619
1392,1,1,1,17.0,0,10.0,10.0,0.0,10.0,1.0,...,0.0,8.0,1.000000,0.000000,21.0,3.0,1.0,21.0,0.142857,0.047619
1393,2,1,1,17.0,0,11.0,11.0,0.0,11.0,1.0,...,0.0,7.0,1.000000,0.000000,21.0,3.0,1.0,21.0,0.142857,0.047619


In [8]:
y=dyslexia_df1['Dyslexia']
X=dyslexia_df1.loc[:, dyslexia_df1.columns != 'Dyslexia']

from sklearn.model_selection import train_test_split
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Using NN to Train the model

In [9]:
# using Neural network to train the model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=16)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'NN model accuracy score: {accuracy}')

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


237/237 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8737 - loss: 0.3841
Epoch 2/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8992 - loss: 0.2602
Epoch 3/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9074 - loss: 0.2305
Epoch 4/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9188 - loss: 0.2042
Epoch 5/5
237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9292 - loss: 0.1839
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9003 - loss: 0.2983
NN model accuracy score: 0.9031745791435242


# Using Self Supervised Learning Approach

In [10]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

**Create the custom dataset**
Generate an augmented version of data using random function

In [11]:
class DyslexiaDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # In SSL, we create augmented versions of data (self-supervision)
        x = self.data[idx]
        augmented_x = self.augment_data(x)
        return augmented_x, x

    def augment_data(self, x):
        # Add Gaussian noise as augmentation
        noise = np.random.normal(0, 0.1, x.shape)
        return x + noise

**Defining a self supervised model**

In [12]:
class SSLModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SSLModel, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)  # Reconstruct the original input
        )

    def forward(self, x):
        x = x.float()  # Ensure input is in float32
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

**Pretrain using Self Supervised Learning**

In [13]:
def train_ssl(model, dataloader, epochs=10, lr=1e-3):
    criterion = nn.MSELoss()  # Using Mean Squared Error loss as it is reconstruction
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        total_loss = 0
        for inputs, _ in dataloader:
            # Ensure the input is in float32
            inputs = inputs.float().to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs, _ = model(inputs)

            # Calculate loss (inputs should be float32 as well)
            loss = criterion(outputs, inputs)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}')

**Fine tuning on Labelled data**

In [14]:
class Classifier(nn.Module):
    def __init__(self, pretrained_encoder, hidden_size, num_classes):
        super(Classifier, self).__init__()
        self.encoder = pretrained_encoder.encoder  # Use the pretrained encoder
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        output = self.classifier(encoded)
        return output

def train_classifier(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}')

**model evaluation**

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
def evaluate_model(model, X_test, y_test):
    model.eval()  # Set the model to evaluation mode

    # Convert the test data to NumPy arrays before creating PyTorch tensors
    X_test_array = X_test.values  # Convert DataFrame to NumPy array
    y_test_array = y_test.values  # Convert Series to NumPy array

    X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test_array, dtype=torch.long).to(device)

    # Forward pass to get predictions
    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predictions = torch.max(outputs, 1)

    # Move predictions and labels to CPU for metrics calculation
    predictions = predictions.cpu().numpy()
    y_test = y_test_tensor.cpu().numpy()

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')
    conf_matrix = confusion_matrix(y_test, predictions)

    # Print out the metrics
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-Score: {f1:.4f}')

**Execution begins**

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Convert to numpy arrays first
X_train_array = X_train.values  # Convert DataFrame to NumPy array
X_test_array = X_test.values  # Convert DataFrame to NumPy array
y_train_array = y_train  # Already a Series, no need to convert
y_test_array = y_test  # Already a Series, no need to convert


# Now convert to torch tensors and ensure float32 dtype
X_train_tensor = torch.tensor(X_train_array, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_array, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_array.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_array.values, dtype=torch.long)

# SSL pretraining
ssl_dataset = DyslexiaDataset(X_train_tensor)
ssl_dataloader = DataLoader(ssl_dataset, batch_size=16, shuffle=True)

ssl_model = SSLModel(input_size=X_train.shape[1], hidden_size=128).to(device)
train_ssl(ssl_model, ssl_dataloader)


# Fine-tune the model
classifier_model = Classifier(pretrained_encoder=ssl_model, hidden_size=128, num_classes=len(set(y))).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier_model.parameters(), lr=1e-3)

train_dataloader = DataLoader(list(zip(X_train_tensor, y_train_tensor)), batch_size=32, shuffle=True)
train_classifier(classifier_model, train_dataloader, criterion, optimizer)

Epoch 1/10, Loss: 683.5923
Epoch 2/10, Loss: 206.2242
Epoch 3/10, Loss: 109.3971
Epoch 4/10, Loss: 75.0462
Epoch 5/10, Loss: 50.3247
Epoch 6/10, Loss: 42.1957
Epoch 7/10, Loss: 41.3592
Epoch 8/10, Loss: 36.7229
Epoch 9/10, Loss: 27.1199
Epoch 10/10, Loss: 32.8768
Epoch 1/10, Loss: 0.8678609579958084
Epoch 2/10, Loss: 0.5041194110482681
Epoch 3/10, Loss: 0.3958955345158817
Epoch 4/10, Loss: 0.34546993403625087
Epoch 5/10, Loss: 0.32333332469232945
Epoch 6/10, Loss: 0.3246013434494243
Epoch 7/10, Loss: 0.3042879394498192
Epoch 8/10, Loss: 0.2978860681735668
Epoch 9/10, Loss: 0.2956427869488712
Epoch 10/10, Loss: 0.289294266400217


In [17]:
evaluate_model(classifier_model, X_test, y_test)

Accuracy: 0.9016
Precision: 0.8639
Recall: 0.9016
F1-Score: 0.8653
